# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [160]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.'''

## 2. Ask for parameters

In [184]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [185]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Sets:

* **I**: Set of potential facility locations.
* **J**: Set of customers. 

## Parameters:

* **f<sub>i</sub>**: Fixed annual activation cost of facility *i* ∈ I.
* **C<sub>i</sub>**: Maximum annual service volume of facility *i* ∈ I.
* **d<sub>j</sub>**: Annual demand of customer *j* ∈ J.
* **t<sub>ij</sub>**: Transportation cost of servicing customer *j* ∈ J from facility *i* ∈ I.

## Decision Variables:

* **x<sub>ij</sub>**: Amount of demand of customer *j* ∈ J  serviced by facility *i* ∈ I.
* **y<sub>i</sub>**: Binary variable equal to 1 if facility *i* ∈ I is opened, and 0 otherwise. 


# 2. Ask for objective

In [186]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [187]:
Markdown(response2.text)

Minimize:  ∑<sub>i∈I</sub> (f<sub>i</sub> * y<sub>i</sub>) + ∑<sub>i∈I</sub>∑<sub>j∈J</sub> (t<sub>ij</sub> * x<sub>ij</sub>) 


# 3. Ask for constraints

In [188]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [189]:
Markdown(response3.text)

Subject to:

1. **Demand fulfillment:** ∑<sub>i∈I</sub> x<sub>ij</sub> = d<sub>j</sub>,  ∀ j ∈ J 
2. **Capacity constraint:** ∑<sub>j∈J</sub> x<sub>ij</sub> ≤ C<sub>i</sub> * y<sub>i</sub>,  ∀ i ∈ I
3. **Service only from open facility:** x<sub>ij</sub> ≤ C<sub>i</sub> * y<sub>i</sub>,  ∀ i ∈ I, j ∈ J
4. **Binary variable and non-negativity**:  y<sub>i</sub> ∈ {0,1}, x<sub>ij</sub> ≥ 0,  ∀ i ∈ I, j ∈ J 


## 3. Generate the pyomo code

In [194]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [195]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with your actual data)
I = ['Facility_1', 'Facility_2', 'Facility_3']  # Set of potential facility locations
J = ['Customer_1', 'Customer_2', 'Customer_3', 'Customer_4']  # Set of customers

f = {'Facility_1': 100, 'Facility_2': 150, 'Facility_3': 120}  # Fixed annual activation cost
C = {'Facility_1': 200, 'Facility_2': 250, 'Facility_3': 180}  # Maximum annual service volume
d = {'Customer_1': 80, 'Customer_2': 120, 'Customer_3': 50, 'Customer_4': 60}  # Annual demand
t = {  # Transportation cost
    ('Facility_1', 'Customer_1'): 5,
    ('Facility_1', 'Customer_2'): 8,
    ('Facility_1', 'Customer_3'): 6,
    ('Facility_1', 'Customer_4'): 4,
    ('Facility_2', 'Customer_1'): 7,
    ('Facility_2', 'Customer_2'): 3,
    ('Facility_2', 'Customer_3'): 9,
    ('Facility_2', 'Customer_4'): 6,
    ('Facility_3', 'Customer_1'): 4,
    ('Facility_3', 'Customer_2'): 6,
    ('Facility_3', 'Customer_3'): 2,
    ('Facility_3', 'Customer_4'): 8,
}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.I, initialize=f)
model.C = pyo.Param(model.I, initialize=C)
model.d = pyo.Param(model.J, initialize=d)
model.t = pyo.Param(model.I, model.J, initialize=t)

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, domain=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.f[i] * model.y[i] for i in model.I) + sum(
        model.t[i, j] * model.x[i, j] for i in model.I for j in model.J
    )

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_fulfillment(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_fulfillment)

def capacity_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.C[i] * model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_constraint)

def service_from_open_facility(model, i, j):
    return model.x[i, j] <= model.C[i] * model.y[i]
model.service_constraint = pyo.Constraint(model.I, model.J, rule=service_from_open_facility)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver
results = solver.solve(model)

# Print the results
print(results)
print("Total Cost:", pyo.value(model.obj))

# Print which facilities are open and the flow of goods
for i in model.I:
    if pyo.value(model.y[i]) > 0.5:
        print(f"Facility {i} is open.")
        for j in model.J:
            if pyo.value(model.x[i, j]) > 0:
                print(f"   Serving {pyo.value(model.x[i, j])} units from {i} to {j}")
```

This code first defines the sample data for sets I, J and parameters f, C, d, and t. **This is where you should replace the sample data with your actual data.**  It then sets up the mathematical model using Pyomo, defines the objective function, and constraints, and finally calls the solver to find the optimal solution. The code then prints the results of the optimization, including the total cost, which facilities should be opened, and how much each customer should receive from each facility. 


## 4. Input problem data and try running the generated code

In [198]:
import pyomo.environ as pyo

# Sample data (replace with your actual data)
# I = ['Facility_1', 'Facility_2', 'Facility_3']  # Set of potential facility locations
# J = ['Customer_1', 'Customer_2', 'Customer_3', 'Customer_4']  # Set of customers

# f = {'Facility_1': 100, 'Facility_2': 150, 'Facility_3': 120}  # Fixed annual activation cost
# C = {'Facility_1': 200, 'Facility_2': 250, 'Facility_3': 180}  # Maximum annual service volume
# d = {'Customer_1': 80, 'Customer_2': 120, 'Customer_3': 50, 'Customer_4': 60}  # Annual demand
# t = {  # Transportation cost
#     ('Facility_1', 'Customer_1'): 5,
#     ('Facility_1', 'Customer_2'): 8,
#     ('Facility_1', 'Customer_3'): 6,
#     ('Facility_1', 'Customer_4'): 4,
#     ('Facility_2', 'Customer_1'): 7,
#     ('Facility_2', 'Customer_2'): 3,
#     ('Facility_2', 'Customer_3'): 9,
#     ('Facility_2', 'Customer_4'): 6,
#     ('Facility_3', 'Customer_1'): 4,
#     ('Facility_3', 'Customer_2'): 6,
#     ('Facility_3', 'Customer_3'): 2,
#     ('Facility_3', 'Customer_4'): 8,
# }


I = [1, 2, 3]  # Set of potential facility locations
J = [1,2,3,4,5]  # Set of customers

f = {1: 1000, 2: 1000, 3: 1000}  # Fixed annual activation cost
C = {1: 500, 2: 500, 3: 500}   # Maximum annual service volume
d = {1: 80, 2: 270, 3: 250,4:160,5:180}  # Annual demand
t = {
    (1, 1): 4,  (2, 1): 6,  (3, 1): 9,
    (1, 2): 5,  (2, 2): 4,  (3, 2): 7,
    (1, 3): 6,  (2, 3): 3,  (3, 3): 4,
    (1, 4): 8,  (2, 4): 5,  (3, 4): 3,
    (1, 5): 10, (2, 5): 8,  (3, 5): 4,
}


# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.I, initialize=f)
model.C = pyo.Param(model.I, initialize=C)
model.d = pyo.Param(model.J, initialize=d)
model.t = pyo.Param(model.I, model.J, initialize=t)

# Decision Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, domain=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.f[i] * model.y[i] for i in model.I) + sum(
        model.t[i, j] * model.x[i, j] for i in model.I for j in model.J
    )

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_fulfillment(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_fulfillment)

def capacity_constraint(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.C[i] * model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_constraint)

def service_from_open_facility(model, i, j):
    return model.x[i, j] <= model.C[i] * model.y[i]
model.service_constraint = pyo.Constraint(model.I, model.J, rule=service_from_open_facility)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver
results = solver.solve(model)

# Print the results
print(results)
print("Total Cost:", pyo.value(model.obj))

# Print which facilities are open and the flow of goods
for i in model.I:
    if pyo.value(model.y[i]) > 0.5:
        print(f"Facility {i} is open.")
        for j in model.J:
            if pyo.value(model.x[i, j]) > 0:
                print(f"   Serving {pyo.value(model.x[i, j])} units from {i} to {j}")


Problem: 
- Name: unknown
  Lower bound: 5610.0
  Upper bound: 5610.0
  Number of objectives: 1
  Number of constraints: 23
  Number of variables: 18
  Number of nonzeros: 63
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.03983116149902344
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Total Cost: 5610.0
Facility 2 is open.
   Serving 80.0 units from 2 to 1
   Serving 270.0 units from 2 to 2
   Serving 150.0 units from 2 to 3
Facility 3 is open.
   Serving 100.0 units from 3 to 3
   Serving 160.0 units from 3 to 4
   Serving 180.0 units from 3 to 5


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [199]:
print(response.text)

## Sets:

* **I**: Set of potential facility locations.
* **J**: Set of customers. 

## Parameters:

* **f<sub>i</sub>**: Fixed annual activation cost of facility *i* ∈ I.
* **C<sub>i</sub>**: Maximum annual service volume of facility *i* ∈ I.
* **d<sub>j</sub>**: Annual demand of customer *j* ∈ J.
* **t<sub>ij</sub>**: Transportation cost of servicing customer *j* ∈ J from facility *i* ∈ I.

## Decision Variables:

* **x<sub>ij</sub>**: Amount of demand of customer *j* ∈ J  serviced by facility *i* ∈ I.
* **y<sub>i</sub>**: Binary variable equal to 1 if facility *i* ∈ I is opened, and 0 otherwise. 



In [200]:
print(response2.text)

Minimize:  ∑<sub>i∈I</sub> (f<sub>i</sub> * y<sub>i</sub>) + ∑<sub>i∈I</sub>∑<sub>j∈J</sub> (t<sub>ij</sub> * x<sub>ij</sub>) 



In [201]:
print(response3.text)

Subject to:

1. **Demand fulfillment:** ∑<sub>i∈I</sub> x<sub>ij</sub> = d<sub>j</sub>,  ∀ j ∈ J 
2. **Capacity constraint:** ∑<sub>j∈J</sub> x<sub>ij</sub> ≤ C<sub>i</sub> * y<sub>i</sub>,  ∀ i ∈ I
3. **Service only from open facility:** x<sub>ij</sub> ≤ C<sub>i</sub> * y<sub>i</sub>,  ∀ i ∈ I, j ∈ J
4. **Binary variable and non-negativity**:  y<sub>i</sub> ∈ {0,1}, x<sub>ij</sub> ≥ 0,  ∀ i ∈ I, j ∈ J 



In [202]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with your actual data)
I = ['Facility_1', 'Facility_2', 'Facility_3']  # Set of potential facility locations
J = ['Customer_1', 'Customer_2', 'Customer_3', 'Customer_4']  # Set of customers

f = {'Facility_1': 100, 'Facility_2': 150, 'Facility_3': 120}  # Fixed annual activation cost
C = {'Facility_1': 200, 'Facility_2': 250, 'Facility_3': 180}  # Maximum annual service volume
d = {'Customer_1': 80, 'Customer_2': 120, 'Customer_3': 50, 'Customer_4': 60}  # Annual demand
t = {  # Transportation cost
    ('Facility_1', 'Customer_1'): 5,
    ('Facility_1', 'Customer_2'): 8,
    ('Facility_1', 'Customer_3'): 6,
    ('Facility_1', 'Customer_4'): 4,
    ('Facility_2', 'Customer_1'): 7,
    ('Facility_2', 'Customer_2'): 3,
    ('Facility_2', 'Customer_3'): 9,
    ('Facility_2', 'Customer_4'): 6,
    ('Facility_3', 'Customer_1'): 4,
    ('Facility_3', 'Customer_2'): 6,
    ('Facility_3', 'Customer_3'): 2,
    ('Facility_3', 'Custome